In [1]:
# Import the Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

from citipy import citipy
from config import weather_api_key
from datetime import datetime

In [2]:
# Create a new set of 2,000 random latitudes and longitudes. 
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)


In [3]:
# Add the latitudes and longitudes to a list
coordinates = list (lat_lngs)

In [4]:
# Create list for cities
cities = []

# Identify nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If City is unique, then add to cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities generated: {len(cities)}")

Number of cities generated: 774


In [10]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Metric&APPID=" + weather_api_key

In [11]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        description = city_weather['weather'][0]['description']
        
        # Convert the date to ISO
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information to the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | hobart
Processing Record 2 of Set 1 | hermanus
Processing Record 3 of Set 1 | carnarvon
Processing Record 4 of Set 1 | qaanaaq
Processing Record 5 of Set 1 | isangel
Processing Record 6 of Set 1 | rikitea
Processing Record 7 of Set 1 | bluff
Processing Record 8 of Set 1 | paka
Processing Record 9 of Set 1 | lolua
City not found. Skipping...
Processing Record 10 of Set 1 | hofn
Processing Record 11 of Set 1 | mattru
Processing Record 12 of Set 1 | yusva
City not found. Skipping...
Processing Record 13 of Set 1 | khatanga
Processing Record 14 of Set 1 | port blair
Processing Record 15 of Set 1 | punta arenas
Processing Record 16 of Set 1 | mataura
Processing Record 17 of Set 1 | bay roberts
Processing Record 18 of Set 1 | dubrovnik
Processing Record 19 of Set 1 | vardo
Processing Record 20 of Set 1 | stornoway
Processing Record 21 of Set 1 | tautira
Processing Record 22 of Set 1 | jamestown
Process

In [12]:
len(city_data)

700

In [13]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)



,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hobart,AU,-42.8794,147.3294,13.34,87,100,5.66,overcast clouds
1,Hermanus,ZA,-34.4187,19.2345,15.53,86,7,4.19,clear sky
2,Carnarvon,AU,-24.8667,113.6333,20.04,88,40,3.09,scattered clouds
3,Qaanaaq,GL,77.4840,-69.3632,-24.47,77,3,0.51,clear sky
4,Isangel,VU,-19.5500,169.2667,26.48,81,100,1.12,overcast clouds
5,Rikitea,PF,-23.1203,-134.9692,25.74,70,1,5.76,clear sky
6,Bluff,NZ,-46.6000,168.3333,8.03,86,6,3.10,clear sky
7,Paka,MY,4.6374,103.4368,25.96,94,94,0.00,overcast clouds
8,Hofn,IS,64.2539,-15.2082,7.60,85,12,2.83,few clouds
9,Mattru,SL,7.6244,-11.8332,26.28,72,100,2.97,overcast clouds


In [14]:
# Export the DataFrame as a CSV file, and save it as WeatherPy_Database.csv in the Weather_Database folder
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")